<a href="https://colab.research.google.com/github/yukinaga/minnano_rl/blob/main/section_5/01_lunar_lander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 月面着陸船の制御
深層強化学習を用いて月面着陸船（Lunar Lander）の制御を行います。  
環境はOpenAI Gymのものを使用し、深層強化学習の実装にはStable Baselinesを利用します。

## ライブラリのインストール
Stable Baselinesなどの必要なライブラリをインストールします。  
ランタイムの再起動を求められた場合は、「ランタイム」→「ランタイムを再起動」によりランタイムを再起動します。

In [ ]:
!apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines==2.5.1 box2d box2d-kengz pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!pip install tensorflow==1.14

## ライブラリの導入
OpenAI Gym、Stable Baselinesなどの各ライブラリを設定します。

In [ ]:
import os
import io
import glob
import base64

import gym
from gym.wrappers import Monitor

import numpy as np

from stable_baselines.deepq.policies import MlpPolicy  # 2層のニューラルネットワーク
from stable_baselines.common.vec_env import DummyVecEnv  # ベクトル化環境
from stable_baselines import DQN
from stable_baselines.common.vec_env import VecVideoRecorder

from IPython import display as ipythondisplay
from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore")

## 環境の設定
OpenAI Gymを使って月面着陸船の環境を設定します。

In [ ]:
def env_func():
    return gym.make("LunarLander-v2")

env_vec = DummyVecEnv([env_func])  # 環境のベクトル化が必要

## モデル評価用の関数
DQNのモデルを評価するための関数を用意します。

In [ ]:
def evaluate(env, model, n_step=10000, n_ave=100):

  epi_rewards = [0.0]
  states = env.reset()

  for i in range(n_step):
      action, _h = model.predict(states)  # _hはRNNで使用
      states, rewards, dones, info = env.step(action)
      
      epi_rewards[-1] += rewards[0]  # 最後の要素に累積
      if dones[0]:  # エピソード終了時
          states = env.reset()
          epi_rewards.append(0.0)  # 次のエピソードの報酬

  ave_reward = round(np.average(epi_rewards[:n_ave]), 2)  # 最初の100エピソードで報酬の平均をとる
  return (ave_reward, len(epi_rewards))

## 動画表示用の関数
結果を動画として表示するための関数を用意します。

In [ ]:
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

def show_video(video_dir):
  video_list = glob.glob(video_dir+"/*.mp4")
  if len(video_list) > 0:
    mp4 = video_list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))

## モデルの評価（訓練前）
DQNのモデルを設定し、訓練前に評価します。  
訓練前なので、月面着陸船はまともに着陸することはできません。


In [ ]:
env = VecVideoRecorder(env_vec, video_folder="videos_before_train/",  # 動画記録の設定
                            record_video_trigger=lambda step: step == 0, video_length=500,
                            name_prefix="")

model = DQN(MlpPolicy, env, verbose=0)  # DQNの設定

ave_reward, n_episode = evaluate(env, model, n_step=10000, n_ave=100)  # モデルの評価
print("ave_reward:", ave_reward, "n_episode:", n_episode)

この時点での動作を動画で確認します。

In [ ]:
show_video("videos_before_train")

## モデルの訓練
月面探索船が正しく着陸できるように、モデルを訓練します。  
訓練済みのモデルは、いつでも利用できるように保存しておきます。

In [ ]:
trained_model = DQN(MlpPolicy, env_vec, verbose=0)  # モデルの初期化

trained_model.learn(total_timesteps=100000)  # モデルの訓練
trained_model.save("lunar_lander_control")  # モデルの保存

## 訓練済みモデルの評価
学習が上手く進めば、月面着陸船は適切に着陸できるようになります。

In [ ]:
env = VecVideoRecorder(env_vec, video_folder="videos_after_train/",  # 動画記録の設定
                            record_video_trigger=lambda step: step == 0, video_length=500,
                            name_prefix="")

ave_reward, n_episode = evaluate(env, trained_model, n_step=10000, n_ave=100)  # モデルの評価
print("ave_reward:", ave_reward, "n_episode:", n_episode)

この時点での動作を動画で確認します。

In [ ]:
show_video("videos_after_train")